# Data Source
https://docs.google.com/spreadsheets/d/1irg60f9qsZOkhp0cwOU7Cy4rJQeyusEUzTNQzhoTYTU/edit?gid=0#gid=0

Let's sanitize this into a nice JSON file to use as a data source for the website.


In [1]:
import { DeviceParser } from "./device.parser.ts";

// parse handhelds html 
const parsedDevicesHandhelds = DeviceParser.parseHandheldsHtml("./Handhelds.html");
console.info("parsedDevicesHandhelds: ", parsedDevicesHandhelds.length);

const parsedDevicesOEMs= DeviceParser.parseOEMsHtml("./OEM.html");
console.info("parsedDevicesOEMs: ", parsedDevicesOEMs.length);

const allDevices = [...parsedDevicesHandhelds, ...parsedDevicesOEMs];
console.info("allDevices: ", allDevices.length);
const devices = allDevices
.filter((device) => device.name.sanitized !== "")
.filter((device) => device.image.originalUrl !== "");


parsedDevicesHandhelds:  438
parsedDevicesOEMs:  24


In [2]:
// marshall into JSON and into bytes
const devicesJson = JSON.stringify(devices, null, 2);
const devicesJsonBytes = new TextEncoder().encode(devicesJson);

In [3]:
// place source/handhelds.json
const filePath = `results/handhelds.json`;
await Deno.mkdir('results', { recursive: true });
await Deno.writeFile(filePath, devicesJsonBytes);
